In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [18]:
directory_traffic = 'ibb_data_byDay_ANADOLU_AVRUPA'
directory_price = 'petrol_prices'

for cont in ['AVRUPA', 'ANADOLU']:
    temp_df_columns = ['DATE', 'RESULT']
    temp_traffic_df = pd.DataFrame(columns=temp_df_columns)

    start_date = datetime.strptime("2020-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2022-12-31", "%Y-%m-%d")

    current_date = start_date
    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y-%m-%d")
        temp_traffic_df.loc[len(temp_traffic_df)] = [formatted_date, 0.0]
        current_date += timedelta(days=1)
        
    sub_dir = os.path.join(directory_traffic, cont)
    df_price = pd.read_csv(os.path.join(directory_price, f'{cont}.csv'))
    for file in os.listdir(sub_dir):
        df_traffic = pd.read_csv(os.path.join(sub_dir, file))
        temp_traffic_df['RESULT'] += df_traffic['RESULT']
    
    if cont == 'AVRUPA':
        temp_traffic_df['RESULT'] = temp_traffic_df['RESULT']/25
    else:
        temp_traffic_df['RESULT'] = temp_traffic_df['RESULT']/13
        
    for price in ['GASOLINE_PRICE', 'DIESEL_PRICE']:    
        y=temp_traffic_df.drop(temp_traffic_df.index[0])['RESULT']
        X=df_price.drop(df_price.index[[0, -1]])[[price]].values.reshape(-1, 1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        lasso = Lasso(alpha=0.1)
        lasso.fit(X_train, y_train)

        y_pred = lasso.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        print(f"Mean Squared Error between {price[:-6].capitalize()} price and average speed in {cont.capitalize()} is : {mse}")

        coefficients = lasso.coef_
        print(f"Coefficients between {price[:-6].capitalize()} price and average speed in {cont.capitalize()} is : {coefficients}")
    
        print('\n')

Mean Squared Error between Gasoline price and average speed in Avrupa is : 15.058472650392785
Coefficients between Gasoline price and average speed in Avrupa is : [-0.08916213]


Mean Squared Error between Diesel price and average speed in Avrupa is : 14.812052883446855
Coefficients between Diesel price and average speed in Avrupa is : [-0.06651567]


Mean Squared Error between Gasoline price and average speed in Anadolu is : 11.086462599552133
Coefficients between Gasoline price and average speed in Anadolu is : [-0.1100173]


Mean Squared Error between Diesel price and average speed in Anadolu is : 9.25443205765316
Coefficients between Diesel price and average speed in Anadolu is : [-0.09254431]


